# 1 - Imports

In [1]:
import sys
sys.path.append('/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/')

import argparse
import librosa
import librosa.display as ld
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from model_evaluator_2 import ModelEvaluator
from settings import MODEL_EVALUATOR_DEFAULT_SETTINGS

# 2 - Feature extractor class

In [2]:
default_params_dict = MODEL_EVALUATOR_DEFAULT_SETTINGS

default_params_dict['model_checkpoint_path'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/models/v1_tiny/v1_tiny_VGGNL_DoubleMHA_700.chkpt'
default_params_dict['test_clients'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/clients.ndx'
default_params_dict['test_impostors'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/impostors.ndx'
default_params_dict['dump_folder'] = './models_results/v1___'
default_params_dict['data_dir'] = ['/home/usuaris/veussd/DATABASES/VoxCeleb/VoxCeleb1/test/']

default_params = argparse.Namespace(**default_params_dict)

model_evaluator_2 = ModelEvaluator(default_params)

22-10-06 19:23:09 - model_evaluator_2 - INFO - Setting device...
22-10-06 19:23:10 - model_evaluator_2 - INFO - Running on cuda device.
22-10-06 19:23:10 - model_evaluator_2 - INFO - Device setted.
22-10-06 19:23:10 - model_evaluator_2 - INFO - Setting random seed...
22-10-06 19:23:10 - model_evaluator_2 - INFO - Random seed setted.
22-10-06 19:23:10 - model_evaluator_2 - INFO - Loading checkpoint from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/models/v1_tiny/v1_tiny_VGGNL_DoubleMHA_700.chkpt
22-10-06 19:23:11 - model_evaluator_2 - INFO - Model checkpoint was saved at epoch 0
22-10-06 19:23:11 - model_evaluator_2 - INFO - Checkpoint loaded.
22-10-06 19:23:11 - model_evaluator_2 - INFO - Loading data from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/clients.ndx
22-10-06 19:23:11 - model_evaluator_2 - INFO - Loading data from /home/usuaris/veu/federico.costa/git_repositories/DoubleAtt

# 3 - Tests

In [9]:
with torch.no_grad():

    # Switch torch to evaluation mode
    model_evaluator_2.net.eval()

    for model_evaluator_2.batch_number, (input_1, len_1, input_2, len_2, labels) in enumerate(model_evaluator_2.training_generator):

        print(f"Batch {model_evaluator_2.batch_number} of {model_evaluator_2.total_batches}")

        #input_1 = input_1.float().to(model_evaluator_2.device)
        #len_1 = len_1.int().to(model_evaluator_2.device)
        #label = label.int().to(model_evaluator_2.device)

        break

Batch 0 of 1


In [11]:
input_1.size()

torch.Size([36, 3726, 80])

In [12]:
len_1

tensor([ 482,  482,  482,  482, 1478, 1478, 1478, 1478, 1250, 1250, 1250, 1250,
        3726, 3726, 3726, 3726,  642,  642,  642,  642, 1478, 1478, 1478, 1478,
        1250, 1250, 1250, 1250, 3726, 3726, 3726, 3726,  642,  642,  642,  642])

In [13]:
input_2.size()

torch.Size([36, 2190, 80])

In [14]:
len_2

tensor([ 498,  850,  542,  462,  994,  686,  402,  562,  570,  494,  670,  422,
        1062,  482, 1058,  966,  462,  490, 2190,  458,  710,  518,  410,  458,
         878,  678, 1122, 1942,  430, 1106, 1442,  586,  398,  422,  726,  642])

In [15]:
labels

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
1/0

In [ ]:
from data import normalizeFeatures, featureReader, TestDataset

with open(model_evaluator_2.input_params.test_clients, 'r') as clients_file:
    clients_trials = clients_file.readlines()

# Read the paths of the impostors audios
with open(model_evaluator_2.input_params.test_impostors, 'r') as impostors_file:
    impostors_trials = impostors_file.readlines()

# Instanciate a Dataset class
dataset = TestDataset(clients_trials, impostors_trials, model_evaluator_2.params, model_evaluator_2.input_params, model_evaluator_2.net)

In [ ]:
dataset[0]

In [ ]:
input_1.size()

In [ ]:
label.size()

In [ ]:
len_1

In [ ]:
a = np.array([[1,2,3],[1,2,4]])
a.shape

In [ ]:
torch.from_numpy(a)

In [ ]:
torch.tensor(1)

In [ ]:
input_1.size()

In [ ]:
len_1

In [ ]:
label.size()

In [ ]:
label[0].item()

In [ ]:
tensor_example_1 = torch.cat([input_1[0], torch.ones(14, 80).to(model_evaluator_2.device)])
tensor_example_2 = torch.cat([input_1[1], torch.ones(23, 80).to(model_evaluator_2.device)])
tensor_example_3 = torch.cat([input_1[2], torch.ones(11, 80).to(model_evaluator_2.device)])

data = [
    (tensor_example_1, label[0].item(), tensor_example_1.size(0)), 
    (tensor_example_2, label[1].item(), tensor_example_2.size(0)),
    (tensor_example_3, label[2].item(), tensor_example_3.size(0)),
]

features, labels, lengths = zip(*data)

max_len = max(lengths)
# We are assuming that all features have the same number of columns
number_columns = data[0][0].size(1)
padded_features = torch.zeros((len(data), max_len, number_columns))

for i in range(len(data)):
    features_tensor = data[i][0]
    rows, cols = features_tensor.size(0), features_tensor.size(1)
    padded_features_tensor = torch.cat(
        [
            features_tensor, 
            torch.zeros((max_len - rows, cols)).to(model_evaluator_2.device)
        ]
    )
    
    padded_features[i] = padded_features_tensor
    
labels = torch.tensor(labels)
lengths = torch.tensor(lengths)
    
padded_features.size()

In [ ]:
max_len

In [ ]:
lengths

In [ ]:
rows

In [ ]:
a = torch.cat([input_1[0], torch.ones(14, 80).to(model_evaluator_2.device)])
a.size()

In [ ]:
features[0][0]

In [ ]:
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
b



In [ ]:
torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3,2])

In [ ]:
features